<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Description" data-toc-modified-id="Data-Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Description</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Useful-Scripts" data-toc-modified-id="Useful-Scripts-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Useful Scripts</a></span></li><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load the data</a></span></li><li><span><a href="#Data-Processing-and-Feature-Engineering" data-toc-modified-id="Data-Processing-and-Feature-Engineering-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Data Processing and Feature Engineering</a></span><ul class="toc-item"><li><span><a href="#Date-time-features" data-toc-modified-id="Date-time-features-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Date time features</a></span></li><li><span><a href="#Categorical-Features" data-toc-modified-id="Categorical-Features-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Categorical Features</a></span></li></ul></li><li><span><a href="#Missing-Values" data-toc-modified-id="Missing-Values-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Missing Values</a></span></li><li><span><a href="#Save-clean-data" data-toc-modified-id="Save-clean-data-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Save clean data</a></span></li></ul></div>

# Data Description
- Data source: https://www.kaggle.com/AnalyzeBoston/crimes-in-boston

**Context**  
Crime incident reports are provided by Boston Police Department (BPD) to document the initial details surrounding an incident to which BPD officers respond. This is a dataset containing records from the new crime incident report system, which includes a reduced set of fields focused on capturing the type of incident as well as when and where it occurred.

**Content**  
Records begin in June 14, 2015 and continue to September 3, 2018.

**Acknowledgements**  
The data is provided by Analyze Boston. The most up-to-date version can be found [here](https://data.boston.gov/dataset/crime-incident-reports-august-2015-to-date-source-new-system).

**Inspiration**  
- What types of crimes are most common? 
- Where are different types of crimes most likely to occur? 
- Does the frequency of crimes change over the day? Week? Year?

# Imports

In [47]:
import bhishan

In [48]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
autoreload 2

In [50]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(color_codes=True)

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set(context='notebook', style='whitegrid', rc={'figure.figsize': (12,8)})
plt.style.use('ggplot') # better than sns styles.
matplotlib.rcParams['figure.figsize'] = 12,8

import os
import time


# random state
random_state=100
np.random.seed(random_state)

# Jupyter notebook settings for pandas
#pd.set_option('display.float_format', '{:,.2g}'.format) # numbers sep by comma
from pandas.api.types import CategoricalDtype
np.set_printoptions(precision=3)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100) # None for all the rows
pd.set_option('display.max_colwidth', 200)

import IPython
from IPython.display import display, HTML, Image, Markdown

print([(x.__name__,x.__version__) for x in [np, pd,sns,matplotlib]])

[('numpy', '1.16.4'), ('pandas', '0.25.0'), ('seaborn', '0.9.0'), ('matplotlib', '3.1.1')]


# Useful Scripts

In [70]:
def show_method_attributes(method, ncols=7,start=None):
    """ Show all the attributes of a given method.
    Example:
    ========
    show_method_attributes(list)
     """
    x = [I for I in dir(method) if I[0]!='_' ]
    x = [I for I in x 
         if I not in 'os np pd sys time psycopg2'.split() ]
    if start:
        x = [I for I in x if I.startswith(start)]

    return pd.DataFrame(np.array_split(x,ncols)).T.fillna('')

# Load the data

In [51]:
df = pd.read_csv('../data/raw/crime.csv',encoding='latin-1')
print(df.shape)
df.head()

(319073, 17)


,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [52]:
df.columns

Index(['INCIDENT_NUMBER', 'OFFENSE_CODE', 'OFFENSE_CODE_GROUP',
       'OFFENSE_DESCRIPTION', 'DISTRICT', 'REPORTING_AREA', 'SHOOTING',
       'OCCURRED_ON_DATE', 'YEAR', 'MONTH', 'DAY_OF_WEEK', 'HOUR', 'UCR_PART',
       'STREET', 'Lat', 'Long', 'Location'],
      dtype='object')

In [53]:
df.columns = df.columns.str.lower()
df.columns

Index(['incident_number', 'offense_code', 'offense_code_group',
       'offense_description', 'district', 'reporting_area', 'shooting',
       'occurred_on_date', 'year', 'month', 'day_of_week', 'hour', 'ucr_part',
       'street', 'lat', 'long', 'location'],
      dtype='object')

In [54]:
df.columns = ['incident', 'code', 'group',
       'description', 'district', 'area', 'shooting',
       'date', 'year', 'month', 'dayofweek', 'hour', 'ucr_part',
       'street', 'lat', 'long', 'location']
df.head()

,incident,code,group,description,district,area,shooting,date,year,month,dayofweek,hour,ucr_part,street,lat,long,location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


# Data Processing and Feature Engineering

In [55]:
from bhishan.util_ds import get_column_descriptions

## Date time features

In [56]:
from pandas.api.types import CategoricalDtype

In [57]:
# we already have year, month and hour as numeric type
# we need to make occurred on date as datetime object

In [58]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [71]:
show_method_attributes(df['date'].dt)

,0,1,2,3,4,5,6
0,ceil,days_in_month,is_month_end,microsecond,quarter,to_period,week
1,date,daysinmonth,is_month_start,minute,round,to_pydatetime,weekday
2,day,floor,is_quarter_end,month,second,tz,weekday_name
3,day_name,freq,is_quarter_start,month_name,strftime,tz_convert,weekofyear
4,dayofweek,hour,is_year_end,nanosecond,timetz,tz_localize,year
5,dayofyear,is_leap_year,is_year_start,normalize,,,


In [73]:
df['dayofweek']  = df['date'].dt.dayofweek.astype(CategoricalDtype())
df['dayofmonth'] = df['date'].dt.day.astype(CategoricalDtype())
df['dayofyear']  = df['date'].dt.dayofyear.astype(CategoricalDtype())
df['weekofyear'] = df['date'].dt.weekofyear.astype(CategoricalDtype())
df['quarter']    = df['date'].dt.quarter.astype(CategoricalDtype())

df['day_name']   = df['date'].dt.day_name().astype(CategoricalDtype())
df['month_name'] = df['date'].dt.month_name().astype(CategoricalDtype())

df.head()

,incident,code,group,description,district,area,shooting,date,year,month,dayofweek,hour,ucr_part,street,lat,long,location,dayofmonth,dayofyear,weekofyear,quarter,day_name,month_name
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,N,2018-09-02 13:00:00,2018,9,6,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)",2,245,35,3,Sunday,September
1,I182070943,1402,Vandalism,VANDALISM,C11,347,N,2018-08-21 00:00:00,2018,8,1,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)",21,233,34,3,Tuesday,August
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,N,2018-09-03 19:27:00,2018,9,0,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)",3,246,36,3,Monday,September
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,N,2018-09-03 21:16:00,2018,9,0,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)",3,246,36,3,Monday,September
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,N,2018-09-03 21:05:00,2018,9,0,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)",3,246,36,3,Monday,September


## Categorical Features
Ref: https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html

- Some categorical features may be number type, make them string type.
- Some categorical features may be nans make them 'Missing'
- We may fill categorical features by mode.

In [74]:
get_column_descriptions(df)
# 99% shooting data is nans, we can drop this column

,column,dtype,nunique,nans,nans_pct,nzeros,nzeros_pct
0,incident,object,282517,0,0.0%,0,0.0%
1,code,int64,222,0,0.0%,0,0.0%
2,group,object,67,0,0.0%,0,0.0%
3,description,object,244,0,0.0%,0,0.0%
4,district,object,12,1765,0.55%,0,0.0%
5,area,object,879,0,0.0%,0,0.0%
6,shooting,object,2,0,0.0%,0,0.0%
7,date,datetime64[ns],233229,0,0.0%,0,0.0%
8,year,int64,4,0,0.0%,0,0.0%
9,month,int64,12,0,0.0%,0,0.0%


In [75]:
cols_obj = df.select_dtypes(['object']).columns
cols_obj

Index(['incident', 'group', 'description', 'district', 'area', 'shooting',
       'ucr_part', 'street', 'location'],
      dtype='object')

In [76]:
df.select_dtypes(['object']).apply(pd.Series.nunique)

incident       282517
group              67
description       244
district           12
area              879
shooting            2
ucr_part            4
street           4657
location        18194
dtype: int64

In [77]:
len(df)

319073

In [78]:
df.select_dtypes(['object']).apply(pd.Series.nunique).loc[lambda x: x<20]

district    12
shooting     2
ucr_part     4
dtype: int64

In [79]:
cols_obj_small = df.select_dtypes(['object']).apply(pd.Series.nunique).loc[lambda x: x<20].index

for c in cols_obj_small:
    print('\n=========================================')
    print(c) 
    print(df[c].value_counts())


district
B2     49945
C11    42530
D4     41915
A1     35717
B3     35442
C6     23460
D14    20127
E13    17536
E18    17348
A7     13544
E5     13239
A15     6505
Name: district, dtype: int64

shooting
N    318054
Y      1019
Name: shooting, dtype: int64

ucr_part
Part Three    158553
Part Two       97569
Part One       61629
Other           1232
Name: ucr_part, dtype: int64


# Missing Values

In [80]:
# make shootings nans as N
df['shooting'] = df['shooting'].fillna('N')

df['shooting'].value_counts()

N    318054
Y      1019
Name: shooting, dtype: int64

In [81]:
df['lat'] = df['lat'].replace(-1, None)
df['long'] = df['long'].replace(-1, None)

# Save clean data

In [82]:
df.to_csv('../data/processed/data_cleaned.csv',index=False)